In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U langchain-community
!pip install bitsandbytes
!pip install -U bitsandbytes transformers
!pip install faiss-gpu-cu12
!pip install -U langchain-community
!pip install datasets
!pip install rank_bm25
!pip install trl
!pip install kiwipiepy

In [ ]:
import os
from peft import PeftConfig
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
import pandas as pd
import numpy as np
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
data_path = "/content/drive/MyDrive/"

In [ ]:
# #불러오기
# FINETUNED_MODEL = "/content/drive/MyDrive/qlora"

# peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [ ]:
train = pd.read_csv(data_path+"train2.csv" )
test = pd.read_csv(data_path+"test2.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 {row['공종(중분류)']}' 작업에서  '{row['작업프로세스']} 작업프로세스 진행중 {row['인적사고']} 발생 했습니다 "
            f"사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "templeate" : row['templeate'],
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [ ]:
# 훈련 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 {row['공종(중분류)']}' 작업에서  '{row['작업프로세스']} 작업프로세스 진행중 {row['인적사고']} 발생 했습니다 "
            f"사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "templeate" : row['templeate'],
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [ ]:
# # 학습 모델용
# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,                   # 4비트 로드 활성화
#     bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
#     bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
#     bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
# )

In [ ]:
# # 베이스 모델 및 토크나이저 로드
# model = AutoModelForCausalLM.from_pretrained(
#     peft_config.base_model_name_or_path,
#     quantization_config=quant_config,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True
# )
# tokenizer = AutoTokenizer.from_pretrained(
#     peft_config.base_model_name_or_path,
#     trust_remote_code=True
# )


In [ ]:
# # QLoRA 모델 로드
# peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)
# # QLoRA 가중치를 베이스 모델에 병합
# merged_model = peft_model.merge_and_unload()

In [ ]:
############### 조심 ###################

In [ ]:
# 일반 모델용
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

In [ ]:
# bm25 토크나이저로 한국어 토크나이저 사용 하기 위해서 불러옴
from kiwipiepy import Kiwi
kiwi = Kiwi()

def ko_kiwi_tokenizer(text: str):
    # Kiwi 토크나이저는 각 토큰에 대한 다양한 정보를 반환합니다.
    # 여기서는 토큰의 표면 형태(텍스트)만 추출합니다.
    tokens = kiwi.tokenize(text)
    return [token[0] for token in tokens]

In [ ]:
import torch
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document

# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]
docs = [Document(page_content=doc) for doc in train_documents]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)


# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=ko_kiwi_tokenizer , k =3 )

# 앙상블 리트리버 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5]  # 각 리트리버에 동일 가중치 부여 (가중치 합은 1.0)
)




In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample = False,
    #temperature=0.1, # 0~1 까지 0에 가까울 수록 보수적인 답변 즉 창의성이 떨어짐
    return_full_text = False,
    max_new_tokens=128, # 문장 최대 길이 조정
    batch_size=16  # 배치 크기 지정
)

In [ ]:
#- 답변은 한 문장으로 작성하며, 15~30단어 이내로 구성합니다.
# 1. 프롬프트 템플릿 정의 (필요한 입력 변수: templeate, context, question)
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
- 질문에 대해 반드시 단 한 문장으로, 재발 방지 대책 및 향후 조치 계획 만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 주어나 부연 설명, 연결어는 제거하고 핵심 용어만 사용합니다.
- 반드시 실행 명령문 형식으로 작성하고, 구체적인 조치(예: 안전교육, 재교육, 정기점검 등)를 포함해야 합니다.
- 특수문자, 리스트, 목차 등은 사용하지 않습니다.
- 제공된 가이드라인 기반으로 작성하고 참고 문서와 가이드라인에서 사용되는 단어를 활용해서 답변하세요.

### 가이드라인:
{templeate}

### 참고 문서:
{context}

### 질문:
{question}

### 답변:
"""

# 2. PromptTemplate 생성 (input_variables에 "templeate", "context", "question" 모두 포함)
prompt = PromptTemplate(
    input_variables=["templeate", "context", "question"],
    template=prompt_template,
)

# 3. LLM 생성 (이미 정의된 text_generation_pipeline 사용)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
tokenizer.padding_side = "left"

In [ ]:
from torch.utils.data import DataLoader
from datasets import Dataset
import torch
from tqdm import tqdm
import psutil

# Matmul 성능 최적화
torch.set_float32_matmul_precision('high')

# 테스트 데이터 준비
test_questions = combined_test_data['question'].tolist()
test_templeates = combined_test_data['templeate'].tolist()
test_dataset = Dataset.from_dict({
    "question": test_questions,
    "templeate": test_templeates
})

# DataLoader 설정
batch_size = 16
num_workers = 0  # GPU 환경에서는 num_workers=0이 적절함
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

test_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# apply()를 활용한 배치 처리 함수
def process_batch(batch):
    batch_inputs = [
        {"query": q, "templeate": t}
        for q, t in zip(batch["question"], batch["templeate"])
    ]
    batch_outputs = qa_chain.batch(batch_inputs)
    return batch_outputs


# DataLoader를 이용한 배치 처리
for batch in tqdm(test_dataloader, desc="처리 진행률"):
    batch_results = process_batch(batch)
    test_results.extend(batch_results)  # 결과 누적

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))


In [ ]:
# from accelerate import Accelerator
# from torch.utils.data import DataLoader
# from datasets import Dataset
# from tqdm import tqdm

# # Accelerator 객체 생성 (단일 GPU 환경)
# accelerator = Accelerator()

# # 테스트 데이터 준비
# test_questions = combined_test_data['question'].tolist()
# test_dataset = Dataset.from_dict({"question": test_questions})

# # DataLoader 설정 (batch_size, num_workers 등은 환경에 맞게 조정)
# batch_size = 16
# num_workers = 0  # GPU 환경에서는 일반적으로 num_workers=0 사용
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# # Accelerator를 통해 DataLoader 준비 (GPU로 데이터를 전달)
# test_dataloader = accelerator.prepare(test_dataloader)

# test_results = []
# print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# # 배치 처리 함수
# def process_batch(batch):
#     # 각 배치의 "question"에 대해 입력 형식으로 변환
#     batch_inputs = [{"query": q} for q in batch["question"]]
#     batch_outputs = qa_chain.batch(batch_inputs)
#     return batch_outputs

# # DataLoader를 이용한 배치 처리
# for batch in tqdm(test_dataloader, desc="처리 진행률"):
#     batch_results = process_batch(batch)
#     # 단일 GPU 환경이므로 gather() 호출은 생략해도 무방합니다.
#     test_results.extend(batch_results)

# print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

In [ ]:

test_results

In [ ]:
test_results[0]

In [ ]:
import pickle

# test_results를 pickle 파일로 저장
with open("test_results.pkl", "wb") as f:
    pickle.dump(test_results, f)

print("test_results가 'test_results.pkl' 파일에 저장되었습니다.")


In [ ]:
# 'result' 값들만 추출
results = [item['result'] for item in test_results]

# 추출한 결과를 이용해 임베딩 생성
pred_embeddings = embedding.encode(results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

# 최종 결과 저장 (submission의 첫 번째 열에 'result' 값 할당)
submission.iloc[:, 1] = results
submission.iloc[:, 2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('submission-EXAONE-3.5-32B-Instruct.csv', index=False, encoding='utf-8-sig')


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv(data_path+"sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()
# 최종 결과를 CSV로 저장
submission.to_csv('submission-EXAONE-3.5-32B-Instruct.csv', index=False, encoding='utf-8-sig')